In [1]:
import pandas as pd
df = pd.read_csv('flightdata.csv')
df = df.drop('Unnamed: 25', axis=1)
df = df[["MONTH", "DAY_OF_MONTH", "DAY_OF_WEEK", "ORIGIN", "DEST", "CRS_DEP_TIME", "ARR_DEL15"]]
df = df.fillna({"ARR_DEL15":1})


In [2]:
import math
for index, row in df.iterrows():
    df.loc[index,"CRS_DEP_TIME"] = math.floor(row["CRS_DEP_TIME"]/100)
df = pd.get_dummies(df, columns= ['ORIGIN','DEST'])

In [3]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df.drop("ARR_DEL15", axis=1), df['ARR_DEL15'], test_size=0.2, random_state=42)
test_y.shape

(2247,)

In [5]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=13)
model.fit(train_x,train_y)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=13, verbose=0, warm_start=False)

In [6]:
predict = model.predict(test_x)
model.score(test_x,test_y)

0.8602581219403649

In [10]:
from sklearn.metrics import roc_auc_score
prob = model.predict_proba(test_x)
roc_auc_score(test_y,prob[:,1])

0.6743824904998539

In [12]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, predict)

array([[1882,   54],
       [ 260,   51]])

In [17]:
from sklearn.metrics import precision_score
train_predicts = model.predict(train_x)
precision_score(train_y,train_predicts)

0.9972375690607734

In [18]:
from sklearn.metrics import recall_score
recall_score(train_y,train_predicts)

0.8650159744408946